# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier

nltk.download(['punkt', 'stopwords', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [29]:
from sklearn.metrics import precision_recall_fscore_support, classification_report
from sklearn.model_selection import GridSearchCV

In [4]:
# load data from database
engine = create_engine('sqlite:///df_clean.db')
df = pd.read_sql_table('df_clean', engine)
X = df['message']
Y = df.iloc[:,4:]

In [26]:
df

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1


### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    text = re.sub(r'^[a-zA-Z0-9]', " ", text.lower())
    token = word_tokenize(text)
    
    token = [w for w in token if w not in stopwords.words('english')]
    
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in token:
        clean_token = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_token)
    
    
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [24]:
from sklearn.multioutput import MultiOutputClassifier
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=2019)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [27]:
y_predict = pipeline.predict(X_test)

for i in range(36):
    print (list(Y)[i], '\n', classification_report(y_test.iloc[:,i], y_predict[:,i]))

related 
              precision    recall  f1-score   support

          0       0.65      0.38      0.48      1883
          1       0.82      0.93      0.87      5921
          2       0.28      0.18      0.22        61

avg / total       0.78      0.79      0.77      7865

request 
              precision    recall  f1-score   support

          0       0.89      0.98      0.93      6533
          1       0.81      0.42      0.55      1332

avg / total       0.88      0.89      0.87      7865

offer 
              precision    recall  f1-score   support

          0       0.99      1.00      1.00      7821
          1       0.00      0.00      0.00        44

avg / total       0.99      0.99      0.99      7865

aid_related 
              precision    recall  f1-score   support

          0       0.75      0.86      0.80      4629
          1       0.74      0.59      0.66      3236

avg / total       0.75      0.75      0.74      7865



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


medical_help 
              precision    recall  f1-score   support

          0       0.93      0.99      0.96      7260
          1       0.57      0.10      0.16       605

avg / total       0.90      0.92      0.90      7865

medical_products 
              precision    recall  f1-score   support

          0       0.96      1.00      0.98      7471
          1       0.84      0.12      0.21       394

avg / total       0.95      0.95      0.94      7865

search_and_rescue 
              precision    recall  f1-score   support

          0       0.98      1.00      0.99      7655
          1       0.64      0.10      0.17       210

avg / total       0.97      0.97      0.97      7865

security 
              precision    recall  f1-score   support

          0       0.98      1.00      0.99      7728
          1       0.00      0.00      0.00       137

avg / total       0.97      0.98      0.97      7865

military 
              precision    recall  f1-score   support

          

In [12]:
len(list(Y))

36

### 6. Improve your model
Use grid search to find better parameters. 

In [50]:
parameters = {'clf__estimator__max_depth': [5, 10, 15]}

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [51]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__max_depth': [5, 10, 15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [52]:
y_cv_pred = cv.predict(X_test)
for i in range(36):
    print (list(Y)[i], '\n', classification_report(y_test.iloc[:,i], y_cv_pred[:,i]))

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


related 
              precision    recall  f1-score   support

          0       0.00      0.00      0.00      1883
          1       0.75      1.00      0.86      5921
          2       0.00      0.00      0.00        61

avg / total       0.57      0.75      0.65      7865

request 
              precision    recall  f1-score   support

          0       0.83      1.00      0.91      6533
          1       0.00      0.00      0.00      1332

avg / total       0.69      0.83      0.75      7865

offer 
              precision    recall  f1-score   support

          0       0.99      1.00      1.00      7821
          1       0.00      0.00      0.00        44

avg / total       0.99      0.99      0.99      7865

aid_related 
              precision    recall  f1-score   support

          0       0.60      1.00      0.75      4629
          1       0.88      0.04      0.07      3236

avg / total       0.71      0.60      0.47      7865

medical_help 
              precision    reca

In [49]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [54]:
import pickle
filename = 'classifier.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.